In [31]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import time
import threading  # Import threading for the timer
import requests
from bs4 import BeautifulSoup
import json
from jupyter_ui_poll import ui_events

In [32]:
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return


def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=10)
    text_input.disabled = True
    return event['description']

In [33]:
output = widgets.Output()
game_start_time = time.time()
# Initialize global variables for game state
points = 0
current_level = 1
game_over = False
game_start = False

# Define cube images and correct answers
cube_images = {
    1: Image("Cube1.png", width=550),
    2: Image("Cube2.png", width=550),
    3: Image("Cube3.png", width=550),
    4: Image("Cube4.png", width=550),
    5: Image("Cube5.png", width=550),
    6: Image("Cube6.png", width=550),
    7: Image("Cube7.png", width=550),
    8: Image("Cube8.png", width=550),
    9: Image("Cube9.png", width=550),
    10: Image("Cube10.png", width=550),
    11: Image("Cube11.png", width=550),
    12: Image("Cube12.png", width=550),
}

correct_answers = {
    1: 'd',
    2: 'b',
    3: 'c',
    4: 'b',
    5: 'd',
    6: 'd',
    7: 'b',
    8: 'a',
    9: 'b',
    10: 'a',
    11: 'd',
    12: 'a',
}
print("Enter your anonymised ID")
print("")
print("To generate an anonymous 4-letter unique user identifier please enter:")
print("- two letters based on the initials (first and last name) of a childhood friend")
print("- two letters based on the initials (first and last name) of a favourite actor / actress")
print("e.g. if your friend was called Charlie Brown and film star was Tom Cruise")
print("then your unique identifier would be CBTC ")
name = input()
while len(name.strip()) == 0:
    print("Invalid input! Name cannot be empty.")
    name = input("Please enter your name again: ")
time.sleep(1)
clear_output(wait=False)


print("Please enter your age:")
age = input()
while len(str(age)) == 0:
    print("Invalid input! Age cannot be empty.")
    age = input("Please enter your age again: ")
if int(age) < 0:
    print("Invalid input! Age cannot be negative.")
    age = input("Please enter your age again: ")
time.sleep(1)
clear_output(wait=False)

print("What is your ethnicity?")

# Create buttons with options
btna1 = widgets.Button(description='White')
btna2 = widgets.Button(description='Black or African American')
btna3 = widgets.Button(description='Asian')
btna4 = widgets.Button(description='Other')
btna5 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btna1.on_click(register_event) 
btna2.on_click(register_event) 
btna3.on_click(register_event) 
btna4.on_click(register_event) 
btna5.on_click(register_event) 

display(btna1, btna2, btna3, btna4, btna5)
event_infoa = wait_for_event(timeout=60)

ethnicity = event_infoa['description']
print(f"User clicked: {event_infoa['description']}")

time.sleep(1)
clear_output(wait=True)


print("Please enter your biological sex:")

btnb1 = widgets.Button(description='Male')
btnb2 = widgets.Button(description='Female')
btnb3 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnb1.on_click(register_event) 
btnb2.on_click(register_event) 
btnb3.on_click(register_event) 

display(btnb1, btnb2, btnb3)
event_infob = wait_for_event(timeout=60)

sex = event_infob['description']
print(f"User clicked: {event_infob['description']}")

time.sleep(1)
clear_output(wait=True)


print("Did you have breakfast today:")
btnc1 = widgets.Button(description='Yes')
btnc2 = widgets.Button(description='No')
btnc3 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnc1.on_click(register_event) 
btnc2.on_click(register_event) 
btnc3.on_click(register_event) 

display(btnc1, btnc2, btnc3)
event_infoc = wait_for_event(timeout=60)

breakfast = event_infoc['description']
print(f"User clicked: {event_infoc['description']}")

time.sleep(1)
clear_output(wait=True)

print("How long did you sleep last night?")
btnd1 = widgets.Button(description='less than 5 hours')
btnd2 = widgets.Button(description='5-9 hours')
btnd3 = widgets.Button(description='more than 9 hours')
btnd4 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnd1.on_click(register_event) 
btnd2.on_click(register_event) 
btnd3.on_click(register_event) 
btnd4.on_click(register_event) 

display(btnd1, btnd2, btnd3, btnd4)
event_infod = wait_for_event(timeout=60)

sleep = event_infod['description']
print(f"User clicked: {event_infod['description']}")

time.sleep(1)
clear_output(wait=True)

data = {
    'Name': name,
    'Age': age,
    'Ethnicity': ethnicity,
    'Sex': sex,
    'Breakfast': breakfast,
    'Sleep': sleep
}
results_json = dict()

# Introductory text widgets using widgets.HTML
Intro = widgets.HTML("<h2>Welcome to the Spatial Reasoning Test</h2>")
Intro2 = widgets.HTML("<h3>Identify a 2D projection which cannot be made by rotating the 3D arrangement.</h3>")
Intro3 = widgets.HTML("<h3>Some arrangements may require you to assume the color by looking at the cube adjacent to it. Please aim for accuracy, not speed.</h3>")

# Group and display introductory elements
intro_box = widgets.VBox([Intro, Intro2, Intro3])
with output:
    display(intro_box)

def check_time_limit():
    global game_over
    if time.time() - game_start_time > 180:  # 180 seconds = 3 minutes
        game_over = True
        end_game()

def update_display(level):
    global start_time
    with output:
        clear_output(wait=True)
        display(widgets.HTML(f"<h3>Level {level}</h3>"))
        display(widgets.HTML(f"<b>Points:</b> {points}"))
        if level in cube_images:
            display(cube_images[level])
        display(widgets.HBox(answer_buttons))
    start_time = time.time()

def on_answer(option, level):
    global current_level, points, start_time, data_dic
    time_used = time.time() - start_time
    correct = option == correct_answers[level]
    data[f'Is question {level} correct'] = "Yes" if correct else "No"
    results_json[f'Is question {level} correct'] = "Yes" if correct else "No"
    results_json[f'Question {level} time'] = f"{time_used:.2f}s"
    points += 1 if correct else 0
    feedback = "Correct!" if correct else "Incorrect!"
    feedback += f" Point +{1 if correct else 0} | Time used: {time_used:.2f}s"
    with output:
        clear_output(wait=True)
        print(feedback)
        time.sleep(2)
    current_level += 1
    if current_level <= len(correct_answers):
        update_display(current_level)
    else:
        end_game()
def on_button_click(b):
    if not game_over:
        check_time_limit()  # Check if time limit reached before processing the answer
        if not game_over:
            on_answer(b.description.lower(), current_level)

# Initialize and configure answer buttons
answer_buttons = [widgets.Button(description=answer) for answer in ['A', 'B', 'C', 'D']]
for button in answer_buttons:
    button.on_click(on_button_click)

start_button = widgets.Button(description='Start Test')
def start_test(b):
    global current_level, points, game_over, game_start_time
    with output:
        clear_output(wait=False)
    game_start_time = time.time()  # Reset game start time each time the test starts
    current_level = 1
    points = 0
    game_over = False
    check_time_limit()  # Initial check to see if we're already past the time limit
    if not game_over:
        update_display(current_level)
    
start_button.on_click(start_test)
with output:
    display(start_button)
display(output)  # Ensure output is displayed for feedback

# Timer function that waits for 3 minutes before ending the game
def timer_thread():
    global game_over
    time.sleep(180)  # Wait for 180 seconds (3 minutes)
    if not game_over:  # Check if the game is not already over
        display(output)
        with output:
            clear_output(wait=True)
            display(widgets.HTML("<h3>Time's up! Ending the test.</h3>"))
        time.sleep(2)  # Give a brief moment to show the time's up messageend_game
        end_game()
def end_game():
    global game_over
    game_over = True
    # Calculate the percentage score
    percentage_score = (points / len(correct_answers)) * 100
    data[f'Correct rate'] = f'{percentage_score:.2f}%'
    results_json[f'Correct rate'] = f'{percentage_score:.2f}%'
    data["results_json"] =json.dumps(results_json)
    with output:
        clear_output(wait=True)
        display(widgets.HTML(f"<h3>Times up Thank You for Participating! You scored {points} points out of {len(correct_answers)}.</h3>"))
        display(widgets.HTML(f"<h3>This gives you a score of {percentage_score:.2f}%.</h3>"))
        print("\nPlease read:")
        print("")
        print("We wish to record your response data to an anonymised public data repository. ")
        print("Your data will be used for educational teaching purposes practising data analysis and visualisation.")
        print("")
        print("Please type   yes   in the box below if you consent to the upload.")
        result = text_input("> ")
        print("User Entered", result)
        if result.lower() == "yes":
            print("Thanks for your participation.")
            print("Please contact a.fedorec@ucl.ac.uk")
            print("If you have any questions or concerns")
            print("regarding the stored results.")
            form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSc09g4wn9F-PQvVf4nKFeSIXgAiZwbFqKEps_2PzByODtAKig/viewform'
            send_to_google_form(data,form_url)
        else:
            #end code execution by raising an exception
            raise(Exception("User did not consent to continue test."))
        clear_output(wait=True)

Output()